In [1]:
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
#model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=658).to(device)
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=314).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model3004/pytorch_model.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [2]:
def predict_class(text):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        return Label_encoder.inverse_transform([predicted_class_id])

In [14]:
predict_class('Автомобильные покрышки резиновые новые')[0]

'4011'

In [2]:
def predict_prob(text, qtty=2):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [3]:
print(predict_prob('бананы в коробках', 5))

{'0808': 0.8409392833709717, '0805': 0.014152244664728642, '2007': 0.006393834948539734, '6702': 0.005769975483417511, '2106': 0.005318484269082546}


In [31]:
preds = predict_prob("гелевая ручка", 5)
print('Code', 'prob-ty', sep=':    ')
[print(i, f"{preds[i]:.{5}f}", sep=':    ') for i in preds]


Code:    prob-ty
8477:    0.27823
3926:    0.13464
6406:    0.03673
9608:    0.01928
3923:    0.01838


[None, None, None, None, None]

In [38]:
def consolelog(probs):
    print('Code', 'prob-ty', sep=':|    ')
    print('_________________')
    for each in probs:
        print(each, f"{probs[each]:.{5}f}", sep=':|    ')
        print('_________________')

In [40]:
consolelog(predict_prob("шины новые", 10))

Code:|    prob-ty
_________________
4011:|    0.99986
_________________
8714:|    0.00001
_________________
8716:|    0.00001
_________________
4012:|    0.00001
_________________
8483:|    0.00000
_________________
8708:|    0.00000
_________________
4009:|    0.00000
_________________
8409:|    0.00000
_________________
4810:|    0.00000
_________________
8419:|    0.00000
_________________


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [17]:
len(Label_encoder.classes_)

314

In [4]:
model.to(device)
from datasets import load_dataset, load_metric, set_caching_enabled
set_caching_enabled(False)
dataset = load_dataset('csv', data_files={'train': '../../train.csv', 'test': '../../test.csv'})
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=1)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=1)
from tqdm import tqdm
#from sklearn import preprocessing
from sklearn.metrics import classification_report
#Label_encoder = preprocessing.LabelEncoder()
#Label_encoder.classes_ = np.load('./classes.npy', allow_pickle=True)



C:\Users\yeril\AppData\Local\Temp\ipykernel_12380\3618075809.py:3: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(False)


Extracting data files: 100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 249.09it/s]


Dataset csv downloaded and prepared to C:/Users/yeril/.cache/huggingface/datasets/csv/default-35c82bc51b408539/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 189.23it/s]


In [5]:
true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(torch.device('cuda')) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

100%|████████████████████████████████████████████████████████████████████████████| 17209/17209 [16:17<00:00, 17.61it/s]
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        0805       1.00      1.00      1.00         9
        0808       1.00      1.00      1.00        16
        1704       0.99      0.99      0.99        80
        1905       1.00      1.00      1.00        14
        2003       1.00      1.00      1.00        15
        2007       1.00      1.00      1.00         7
        2009       1.00      1.00      1.00         8
        2105       1.00      1.00      1.00         8
        2106       0.84      0.90      0.87        30
        2202       0.60      1.00      0.75         3
        2710       1.00      1.00      1.00         8
        2918       0.76      0.90      0.83        29
        2924       0.71      0.81      0.76        27
        2933       0.68      0.75      0.71        55
        2934       0.81      0.69      0.75        36
        2941       0.95      0.86      0.90        22
        3005       1.00      1.00      1.00        14
        3206       0.88    

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
